In [23]:
import rasterio
from rasterio.features import shapes
from rasterio.plot import show
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import open3d as o3d
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

In [24]:
# gdal_calc.py -A DSM/GeoTIFF/DHMVII_DSM_1m_testdata.tif -B ./dtm_1m.tif --outfile=CHM.tif --calc="A-B" --NoDataValue=0
# gdalwarp  DTM/GeoTIFF/DHMVII_DTM_5m_testdata.tif test.tif -tr 1 1 -r near
# gdal_contour -3d -a elev chm.tif contour.shp -i 0.5 -snodata 0b

In [147]:
from pyproj import Transformer
def lambert_to_wgs(x_lambert, y_lambert):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326")
    x_wgs, y_wgs = transformer.transform(x_lambert, y_lambert)
    return x_wgs, y_wgs

def wgs_to_lambert(x_wgs, y_wgs):
    transformer = Transformer.from_crs("epsg:4326", "epsg:31370")
    x_lambert, y_lambert = transformer.transform(x_wgs, y_wgs)
    return x_lambert, y_lambert

import requests
def search_address(cp, rue, num, as_wgs=False, as_dict=False, boundary=False):    
    url = 'http://geoservices.wallonie.be/geolocalisation/rest/getPositionByCpRueAndNumero/{cp}/{rue}/{num}'
    r = requests.get(url.format(cp=cp, rue=rue, num=num)).json()
    x, y, xMin, xMax, yMin, yMax = r['x'], r['y'], r['rue']['xMin'], r['rue']['xMax'], r['rue']['yMin'], r['rue']['yMax']
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax)
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

def search_address_mapbox(address, as_wgs=False, as_dict=False, boundary=10):
    mapbox_api_key = 'pk.eyJ1IjoiYmFjYXlhdzU1OSIsImEiOiJja2dtNTgyNW8wMWN2MnBzM2loNGt4NmlzIn0.3IYulU7qzfpq2Ms0aaYWMQ'
    g = requests.get(f"https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json?types=address&access_token={mapbox_api_key}")
    if g.status_code != 200:
        return 'NotFound'
    g = g.json()
    y, x = g['features'][0]['center']
    x, y = wgs_to_lambert(x, y)
    xMin, xMax = x - boundary, x + boundary
    yMin, yMax = y - boundary, y + boundary
    
    if as_wgs:
        x, y       = lambert_to_wgs(x, y)
        xMin, xMax = lambert_to_wgs(xMin, xMax)
        yMin, yMax = lambert_to_wgs(yMin, yMax)
        
    if as_dict:
        return {'x': x, 'xMin': xMin, 'xMax': xMax, 
                'y': y, 'yMin': yMin, 'yMax': yMax}
    return x, y, xMin, xMax, yMin, yMax

In [148]:
x, y, xMin, xMax, yMin, yMax = search_address_mapbox("Rue du roton 38 charleroi", as_wgs=False)

In [131]:
x, y, xMin, xMax, yMin, yMax = search_address(cp='5000', rue='RUE DE LA CRETE', num = '111', as_wgs=False)

In [139]:
xMin, xMax, yMin, yMax

(155365.36043714095, 155965.36043714095, 123081.1988786608, 123681.1988786608)

In [68]:
# Christophe Schenke 
# christophe.shanke@spw.walonnie.be  
# 0475 95 4570

In [69]:
#from fiona.transform import transform_geom
#hits[0][1]['geometry']  = transform_geom("epsg:31370", "epsg:4326", hits[0][1]['geometry'])

In [149]:
import fiona
gdb = fiona.open('./BATI3D_2013-2014_FILEGDB_31370/BATI3D_2013-2014.gdb', layer=1)
hits = list(gdb.items(bbox=(xMin, yMin, xMax, yMax)))

In [151]:
hits

[(850701,
  {'type': 'Feature',
   'id': '850701',
   'properties': OrderedDict([('REF_CBE',
                 '9942AE7D-2B32-4437-B54A-658A9566933D'),
                ('APT_ID', None),
                ('ADDR_NUMERO', None),
                ('RUE_NM', None),
                ('COM_INS', None),
                ('COM_NM', None)]),
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[(155662.3769999966,
        123378.90799999982,
        129.13999999999942),
       (155658.72999999672, 123379.5940000005, 129.13999999999942),
       (155656.90600000322, 123373.98699999973, 129.13999999999942),
       (155660.16399999708, 123372.87999999896, 129.13999999999942),
       (155660.37200000137, 123372.80900000036, 129.13999999999942),
       (155661.12399999797, 123375.05299999937, 129.13999999999942),
       (155662.3769999966, 123378.90799999982, 129.13999999999942)]],
     [[(155661.12399999797, 123375.05299999937, 129.13999999999942),
       (155661.12399999797, 123375.05299999937, 1

In [150]:
%%time
import open3d as o3d
meshes = []
for hit in hits:
    points = hit[1]['geometry']['coordinates']
    pts = [item for sublist in points for sitem in sublist for item in sitem]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    mesh = pcd.compute_convex_hull()
    meshes.append(mesh[0])
o3d.visualization.draw_geometries(meshes, mesh_show_wireframe=True, mesh_show_back_face=True)

CPU times: user 771 ms, sys: 521 ms, total: 1.29 s
Wall time: 17.9 s


In [73]:
[item for sublist in hits[0][1]['geometry']['coordinates'] for sitem in sublist for item in sitem]

[(185185.49300000072, 129229.93600000069, 90.10000000000582),
 (185181.67700000107, 129235.54800000042, 90.10000000000582),
 (185173.68400000036, 129230.37999999896, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185185.49300000072, 129229.93600000069, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 100.62810000000172),
 (185185.49300000072, 129229.93600000069, 100.62810000000172),
 (185185.49300000072, 129229.93600000069, 90.10000000000582),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 100.62810000000172),
 (185184.7580000013, 129229.48600000143, 100.62810000000172),
 (185184.7580000013, 129229.48600000143, 90.10000000000582),
 (185177.53299999982, 129224.68699999899, 90.10000000000582),
 (185173.6